In [1]:
!ls ../data

aisles.csv					     order_products__train.csv
departments.csv					     orders.csv
instacart_online_grocery_shopping_2017_05_01.tar.gz  products.csv
order_products__prior.csv


In [2]:
import psycopg2
import pandas as pd
import math

In [3]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="postgres", host="postgres")

In [4]:
cur = conn.cursor()

## Cleaning up database if it is not empty

In [5]:
cur.execute("""SELECT TABLE_NAME FROM information_schema.tables WHERE table_schema = 'public'""")
tables = list(map(lambda x: x[0], cur.fetchall()))

print(tables)

['products', 'orders', 'order_products', 'aisles', 'departments']


In [6]:
if len(tables) > 0:
    cur.execute("DROP TABLE %s;" % ', '.join(tables))
    conn.commit()

## Create database schema

In [7]:
schema = ("""CREATE TABLE departments
(
    id INTEGER PRIMARY KEY,
    name VARCHAR(50)
);

CREATE TABLE aisles
(
    id INTEGER PRIMARY KEY,
    name VARCHAR(50)
);

CREATE TABLE products
(
    id INTEGER PRIMARY KEY,
    name VARCHAR(255),
    aisle_id INTEGER,
    department_id INTEGER,
    CONSTRAINT products_aisles_id_fk FOREIGN KEY (aisle_id) REFERENCES aisles (id),
    CONSTRAINT products_departments_id_fk FOREIGN KEY (department_id) REFERENCES departments (id)
);

CREATE TABLE orders
(
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    eval_set CHAR(5),
    order_number INTEGER,
    day_of_week SMALLINT,
    hour_of_day SMALLINT,
    days_since_prior_order SMALLINT
);

CREATE TABLE order_products
(
    order_id INTEGER,
    product_id INTEGER,
    add_to_cart_order INTEGER,
    reordered BOOLEAN,
    CONSTRAINT order_products_products_id_fk FOREIGN KEY (product_id) REFERENCES products (id),
    CONSTRAINT order_products_orders_id_fk FOREIGN KEY (order_id) REFERENCES orders (id)
);""")
    
cur.execute(schema)
conn.commit()

## Adding content

Add aisles to database

In [8]:
aisles = pd.read_csv('../data/aisles.csv')

for key, row in aisles.iterrows():
    cur.execute("""INSERT INTO aisles(id, name) VALUES(%d, '%s');""" % (row['aisle_id'], row['aisle']))
conn.commit()

Add departments to database

In [9]:
departments = pd.read_csv('../data/departments.csv')

for key, row in departments.iterrows():
    cur.execute("""INSERT INTO departments(id, name) VALUES(%d, '%s');""" % (row['department_id'], row['department']))
conn.commit()

Add products

In [10]:
products = pd.read_csv('../data/products.csv')

for key, row in products.iterrows():
    cur.execute("""INSERT INTO products(id, name, aisle_id, department_id) VALUES(%s, %s, %s, %s);""",
                (row['product_id'], row['product_name'], row['aisle_id'], row['department_id']))
conn.commit()

### Add orders

In [11]:
orders = pd.read_csv('../data/orders.csv')

for key, row in orders.iterrows():
    days_since_prior_order = None
    if not math.isnan(row['days_since_prior_order']):
        days_since_prior_order = row['days_since_prior_order']
    
    cur.execute("""INSERT INTO orders(id, user_id, eval_set, order_number, day_of_week, hour_of_day, days_since_prior_order)
    VALUES(%s, %s, %s, %s, %s, %s, %s)""", (row['order_id'], row['user_id'], row['eval_set'], row['order_number'], row['order_dow'], row['order_hour_of_day'], days_since_prior_order))
conn.commit()

### Add order details

In [12]:
order_products_prior = pd.read_csv('../data/order_products__prior.csv')
order_products_train = pd.read_csv('../data/order_products__train.csv')

In [13]:
set(order_products_prior['order_id']).intersection(set(order_products_train['order_id']))

set()

In [14]:
for dataset in (order_products_prior, order_products_train):
    for key, row in dataset.iterrows():
        cur.execute("""INSERT INTO order_products(order_id, product_id, add_to_cart_order, reordered) VALUES (%s, %s, %s, %s)""",
                   (int(row['order_id']), int(row['product_id']), int(row['add_to_cart_order']), int(row['reordered']) == 1))
conn.commit()

## Validating database row count

In [15]:
cur.execute("""SELECT COUNT(*) FROM aisles""")
cur.fetchone()[0] == len(aisles)

True

In [16]:
cur.execute("""SELECT COUNT(*) FROM departments""")
cur.fetchone()[0] == len(departments)

True

In [17]:
cur.execute("""SELECT COUNT(*) FROM products""")
cur.fetchone()[0] == len(products)

True

In [18]:
cur.execute("""SELECT COUNT(*) FROM orders""")
cur.fetchone()[0] == len(orders)

True

In [19]:
cur.execute("""SELECT COUNT(*) FROM order_products""")
cur.fetchone()[0] == (len(order_products_prior) + len(order_products_train))

True